In [1]:
pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tkinter as tk
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageTk
import math

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Initialize Tkinter GUI
root = tk.Tk()
root.title("Gesture-Based Radial Menu")
root.geometry("500x500")

# Canvas for radial menu
canvas = tk.Canvas(root, width=500, height=500, bg="white")
canvas.pack()

# Text options for radial menu buttons
options = [
    "Elephant", "Monkey", "Cat",
    "Lion", "Dog", "Cow",
    "Duck", "Sheep"
]

# Radial menu settings
center_x, center_y = 250, 250  # Center of the radial menu
radius = 150  # Radius of the circle where options will be placed
num_options = len(options)  # Number of options to place

# Add text options in circular layout
buttons = []
button_backgrounds = []  # Store background color rectangles for each option
for i, option in enumerate(options):
    angle = 2 * math.pi * i / num_options  # Calculate angle for each option
    option_x = center_x + radius * math.cos(angle)
    option_y = center_y + radius * math.sin(angle)
    
    # Background color rectangle (default white)
    rect = canvas.create_oval(option_x - 30, option_y - 30, option_x + 30, option_y + 30, fill="white", outline="")
    button_backgrounds.append(rect)
    
    # Text label for the radial menu option
    button = canvas.create_text(option_x, option_y, text=option, font=("Helvetica", 12))
    buttons.append(button)

# Central volume control button or indicator
canvas.create_oval(center_x - 50, center_y - 50, center_y + 50, center_y + 50, fill="lightgrey")
volume_text = canvas.create_text(center_x, center_y, text="Volume\n62%", font=("Helvetica", 12))

# Create cursor as a small black dot
cursor_radius = 5
cursor = canvas.create_oval(-cursor_radius, -cursor_radius, cursor_radius, cursor_radius, fill="black", outline="")

# OpenCV Video Capture for Gesture Recognition
cap = cv2.VideoCapture(0)

# Gesture detection and interaction function
def detect_gesture():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    # Reset all button backgrounds to white
    for rect in button_backgrounds:
        canvas.itemconfig(rect, fill="white")
    
    if results.multi_hand_landmarks:
        for hand_index, hand_landmarks in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get the index finger tip coordinates for each hand
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            index_coords = (int(index_tip.x * frame.shape[1]), int(index_tip.y * frame.shape[0]))

            # Map OpenCV coordinates to Tkinter canvas coordinates
            canvas_x = int(index_tip.x * 500)
            canvas_y = int(index_tip.y * 500)

            # Update the cursor position
            canvas.coords(cursor, canvas_x - cursor_radius, canvas_y - cursor_radius, 
                          canvas_x + cursor_radius, canvas_y + cursor_radius)

            # Check if the index finger is hovering over any text option
            for i, button in enumerate(buttons):
                button_coords = canvas.coords(button)
                button_x, button_y = button_coords[0], button_coords[1]
                
                # Calculate distance from index finger to button center
                hand_to_button_distance = np.sqrt((button_x - canvas_x) ** 2 + (button_y - canvas_y) ** 2)
                
                if hand_to_button_distance < 30:  # Hovering distance threshold
                    # Change the background color of the section to blue
                    canvas.itemconfig(button_backgrounds[i], fill="lightblue")
                    print(f"Hovering over '{options[i]}'")  # Optional: Print which option is being hovered over

            # Use thumb y-coordinate to control volume (scroll effect)
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            thumb_coords = (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0]))
            
            if thumb_coords[1] < frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n70%")
            elif thumb_coords[1] > 2 * frame.shape[0] // 3:
                canvas.itemconfig(volume_text, text="Volume\n40%")
    
    # Display the frame
    cv2.imshow("Gesture Control", frame)
    cv2.waitKey(1)
    
    # Re-run this function after a short delay
    root.after(10, detect_gesture)

# Start gesture detection loop
root.after(10, detect_gesture)

# Run Tkinter main loop
root.mainloop()

# Release the capture after GUI is closed
cap.release()
cv2.destroyAllWindows()


Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Sheep'
Hovering over 'Monkey'
Hovering over 'Monkey'
